In [1]:
from ultralytics import YOLO
import cv2
import os
import numpy
from typing import List
from pydantic import BaseModel
from PIL import Image
import glob
import csv


In [81]:
import os
import cv2

video_folder = 'C:\\Users\\mikhe\\OneDrive\\Рабочий стол\\диплом\\ddd\\tt'
output_dir = 'tt'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

video_files = os.listdir(video_folder)

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames >= 10:
        frame_step = total_frames // 10
    else:
        frame_step = 1  

    frame_number = 0
    frames_extracted = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            if frame_number % frame_step == 0 and frames_extracted < 10:
                frame_path = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}_frame_{frames_extracted}.jpg")
                cv2.imwrite(frame_path, frame)
                frames_extracted += 1

            frame_number += 1
        else:
            break

    cap.release()


In [2]:

class GetKeypoint(BaseModel):
    NOSE:           int = 0
    LEFT_EYE:       int = 1
    RIGHT_EYE:      int = 2
    LEFT_EAR:       int = 3
    RIGHT_EAR:      int = 4
    LEFT_SHOULDER:  int = 5
    RIGHT_SHOULDER: int = 6
    LEFT_ELBOW:     int = 7
    RIGHT_ELBOW:    int = 8
    LEFT_WRIST:     int = 9
    RIGHT_WRIST:    int = 10
    LEFT_HIP:       int = 11
    RIGHT_HIP:      int = 12
    LEFT_KNEE:      int = 13
    RIGHT_KNEE:     int = 14
    LEFT_ANKLE:     int = 15
    RIGHT_ANKLE:    int = 16

get_keypoint = GetKeypoint()



In [3]:
def extract_keypoint(keypoint):
    # nose
    nose_x, nose_y = keypoint[get_keypoint.NOSE]
    # eye
    left_eye_x, left_eye_y = keypoint[get_keypoint.LEFT_EYE]
    right_eye_x, right_eye_y = keypoint[get_keypoint.RIGHT_EYE]
    # ear
    left_ear_x, left_ear_y = keypoint[get_keypoint.LEFT_EAR]
    right_ear_x, right_ear_y = keypoint[get_keypoint.RIGHT_EAR]
    # shoulder
    left_shoulder_x, left_shoulder_y = keypoint[get_keypoint.LEFT_SHOULDER]
    right_shoulder_x, right_shoulder_y = keypoint[get_keypoint.RIGHT_SHOULDER]
    # elbow
    left_elbow_x, left_elbow_y = keypoint[get_keypoint.LEFT_ELBOW]
    right_elbow_x, right_elbow_y = keypoint[get_keypoint.RIGHT_ELBOW]
    # wrist
    left_wrist_x, left_wrist_y = keypoint[get_keypoint.LEFT_WRIST]
    right_wrist_x, right_wrist_y = keypoint[get_keypoint.RIGHT_WRIST]
    # hip
    left_hip_x, left_hip_y = keypoint[get_keypoint.LEFT_HIP]
    right_hip_x, right_hip_y = keypoint[get_keypoint.RIGHT_HIP]
    # knee
    left_knee_x, left_knee_y = keypoint[get_keypoint.LEFT_KNEE]
    right_knee_x, right_knee_y = keypoint[get_keypoint.RIGHT_KNEE]
    # ankle
    left_ankle_x, left_ankle_y = keypoint[get_keypoint.LEFT_ANKLE]
    right_ankle_x, right_ankle_y = keypoint[get_keypoint.RIGHT_ANKLE]
    
    return [
        nose_x, nose_y,
        left_eye_x, left_eye_y,
        right_eye_x, right_eye_y,
        left_ear_x, left_ear_y,
        right_ear_x, right_ear_y,
        left_shoulder_x, left_shoulder_y,
        right_shoulder_x, right_shoulder_y,
        left_elbow_x, left_elbow_y,
        right_elbow_x, right_elbow_y,
        left_wrist_x, left_wrist_y,
        right_wrist_x, right_wrist_y,
        left_hip_x, left_hip_y,
        right_hip_x, right_hip_y,
        left_knee_x, left_knee_y,
        right_knee_x, right_knee_y,        
        left_ankle_x, left_ankle_y,
        right_ankle_x, right_ankle_y
    ]

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO


def calculate_distance(keypoints1, keypoints2):
    if np.all(keypoints1 == 0) or np.all(keypoints2 == 0):
        return 0
    else:
        dist = np.linalg.norm(keypoints1 - keypoints2)
        return dist

image_folder = './frame'
dataset_csv = []

model = YOLO('yolov8n-pose.pt')
frame = 0
previous_video_name = None  

for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)

    if image_file.startswith('V_'):
        label = 0  
    else:
        label = 1

    video_name = image_file.split('_frame')[0]  

    if previous_video_name != video_name:
        frame = 0
        previous_video_name = video_name

    image = cv2.imread(image_path)

    results = model.predict(image, save=False)[0]
    results_keypoint = results.keypoints.xyn.cpu().numpy()

    keypoints_list = []

    for result_keypoint in results_keypoint:
        if len(result_keypoint) == 17:
            keypoint_list = extract_keypoint(result_keypoint)
            keypoints_list.append(keypoint_list)
            dataset_csv.append([image_file, label, frame] + keypoint_list + [0]) 

    if keypoints_list:
        reference_keypoint = np.array(keypoints_list[0])
        for i in range(1, len(keypoints_list)):
            distance = calculate_distance(reference_keypoint, np.array(keypoints_list[i]))
            dataset_csv[-(len(keypoints_list) - i)][-1] = distance

    frame += 1

for row in dataset_csv:
    print(row)


In [89]:
     
header = [
        'image_file',
        'label',
        'frame',
        'nose_x', 'nose_y',
        'left_eye_x', 'left_eye_y',
        'right_eye_x', 'right_eye_y',
        'left_ear_x', 'left_ear_y',
        'right_ear_x', 'right_ear_y',
        'left_shoulder_x', 'left_shoulder_y',
        'right_shoulder_x', 'right_shoulder_y',
        'left_elbow_x', 'left_elbow_y',
        'right_elbow_x', 'right_elbow_y',
        'left_wrist_x', 'left_wrist_y',
        'right_wrist_x', 'right_wrist_y',
        'left_hip_x', 'left_hip_y',
        'right_hip_x', 'right_hip_y',
        'left_knee_x', 'left_knee_y',
        'right_knee_x', 'right_knee_y',
        'left_ankle_x', 'left_ankle_y',
        'right_ankle_x', 'right_ankle_y',
        'distances'
    ]

print(len(header))



38


In [90]:
with open('kp.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(dataset_csv)

In [85]:
with open('kp.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(dataset_csv)

In [ ]:
import pandas as pd
df = pd.read_csv('kp.csv')
df.head()